In [3]:
import pandas as pd


df = pd.read_excel('C:\General\Code\Docking\REAL.xlsx', sheet_name='Spectra')


In [12]:
df = df[['Previous AA', 'Peptide', 'Protein Name']]
df.dropna(subset=['Previous AA'], inplace=True)

,Previous AA,Peptide,Protein Name
0,E,REPQQPGEKEEDEDEQPRPIPFPRPQPR,>sp|P0DO15|GLCA2_SOYBN Beta-conglycinin alpha ...
1,E,REPQQPGEKEEDEDEQPRPIPFPRPQPR,>sp|P0DO15|GLCA2_SOYBN Beta-conglycinin alpha ...
2,E,REPQQPGEKEEDEDEQPRPIPFPRPQPR,>sp|P0DO15|GLCA2_SOYBN Beta-conglycinin alpha ...
3,E,REPQQPGEKEEDEDEQPRPIPFPRPQPR,>sp|P0DO15|GLCA2_SOYBN Beta-conglycinin alpha ...
4,E,REPQQPGEKEEDEDEQPRPIPFPRPQPR,>sp|P0DO15|GLCA2_SOYBN Beta-conglycinin alpha ...
...,...,...,...
12962,Q,HYFKRAGGYSLCGEWK,>Reverse >tr|K7N1M9|K7N1M9_SOYBN Uncharacteriz...
12963,D,VLSKGGPLK,>Reverse >tr|K7N1M9|K7N1M9_SOYBN Uncharacteriz...
12964,D,VLSKGGPLK,>Reverse >tr|K7N1M9|K7N1M9_SOYBN Uncharacteriz...
12965,D,VLSKGGPLK,>Reverse >tr|K7N1M9|K7N1M9_SOYBN Uncharacteriz...


In [14]:
df['id'] = df['Protein Name'].str.extract(r'\|(\w+)\|')

In [17]:
df = df[['Previous AA', 'Peptide', 'id']]

In [21]:
# Import the necessary modules
import requests

# Create a set to store the IDs for which the sequence has already been retrieved
retrieved_ids = set()

# Retrieve the full protein sequence for each unique ID
def retrieve_full_sequence(id):
    if id in retrieved_ids:
        return None
    else:
        # Using UniProt API to retrieve protein sequences
        url = f"https://www.uniprot.org/uniprot/{id}.fasta"
        response = requests.get(url)
        if response.status_code == 200:
            retrieved_ids.add(id)
            # Process the response to extract the sequence
            sequence = ''.join(response.text.split('\n')[1:])  # Remove the header line
            print(sequence)
            return sequence
        else:
            return None

# Add the full protein sequence to the DataFrame in a new column called "Full_Sequence"
df['Full_Sequence'] = df['id'].apply(retrieve_full_sequence)

...
      Previous AA                       Peptide      id  \
0               E  REPQQPGEKEEDEDEQPRPIPFPRPQPR  P0DO15   
1               E  REPQQPGEKEEDEDEQPRPIPFPRPQPR  P0DO15   
2               E  REPQQPGEKEEDEDEQPRPIPFPRPQPR  P0DO15   
3               E  REPQQPGEKEEDEDEQPRPIPFPRPQPR  P0DO15   
4               E  REPQQPGEKEEDEDEQPRPIPFPRPQPR  P0DO15   
...           ...                           ...     ...   
12962           Q              HYFKRAGGYSLCGEWK  K7N1M9   
12963           D                     VLSKGGPLK  K7N1M9   
12964           D                     VLSKGGPLK  K7N1M9   
12965           D                     VLSKGGPLK  K7N1M9   
12966           P        FVDVNFLDKYPVSPLSEDYTEH  K7N1M9   

                                           Full_Sequence  
0      MMRARFPLLLLGLVFLASVSVSFGIAYWEKENPKHNKCLQSCNSER...  
1                                                   None  
2                                                   None  
3                                                  

In [25]:
df['Full_Sequence'] = df.groupby('id')['Full_Sequence'].fillna(method='ffill')


C:\Windows\Temp\ipykernel_35576\1825484934.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Full_Sequence'] = df.groupby('id')['Full_Sequence'].fillna(method='ffill')


In [27]:
df.drop_duplicates(subset=['Peptide'], inplace=True)

In [32]:
# Function to extract the desired sequence
def extract_sequence(row):
    # Concatenate Previous AA with the first 5 amino acids of Peptide
    search_string = row['Previous AA'] + row['Peptide'][:5]

    # Find the concatenated string within the Full_Sequence
    index = row['Full_Sequence'].find(search_string)

    # If found, extract the sequence including 4 amino acids before Previous AA
    if index != -1 and index >= 4:
        return row['Full_Sequence'][index-4:index+6]  # 4 amino acids before + 6 amino acids (Previous AA + first 5 of Peptide)
    else:
        return None

# Apply the function to each row
df['Extracted_Sequence'] = df.apply(extract_sequence, axis=1)


In [34]:
df.dropna(subset=['Extracted_Sequence'], inplace=True)

In [39]:
df.drop_duplicates(subset=['Extracted_Sequence'], inplace=True)

In [40]:
df

,Previous AA,Peptide,id,Full_Sequence,Extracted_Sequence
0,E,REPQQPGEKEEDEDEQPRPIPFPRPQPR,P0DO15,MMRARFPLLLLGLVFLASVSVSFGIAYWEKENPKHNKCLQSCNSER...,PQHPEREPQQ
12,K,EEDEDEQPRPIPFPRPQPR,P0DO15,MMRARFPLLLLGLVFLASVSVSFGIAYWEKENPKHNKCLQSCNSER...,QPGEKEEDED
13,E,DEDEQPRPIPFPRPQPR,P0DO15,MMRARFPLLLLGLVFLASVSVSFGIAYWEKENPKHNKCLQSCNSER...,GEKEEDEDEQ
32,D,EDEQPRPIPFPRPQPR,P0DO15,MMRARFPLLLLGLVFLASVSVSFGIAYWEKENPKHNKCLQSCNSER...,EKEEDEDEQP
42,E,DEQPRPIPFPRPQPR,P0DO15,MMRARFPLLLLGLVFLASVSVSFGIAYWEKENPKHNKCLQSCNSER...,KEEDEDEQPR
...,...,...,...,...,...
12946,K,LLEFD,K7M4I0,MEDMDEIDAVPQVYMACILNGNRIGVSYYDSTLRQLYVLEVWDDGD...,DAMEKLLEFD
12948,K,IFFE,K7M4I0,MEDMDEIDAVPQVYMACILNGNRIGVSYYDSTLRQLYVLEVWDDGD...,KGDLKIFFED
12957,A,GLDAKPTVL,I1NFX7,MATATSQTLRFQSPSLSLSSKLPLSSAFSVSLRPRHRGSRPLVLVV...,LVVSAGLDAK
12959,R,IVNV,I1NFX7,MATATSQTLRFQSPSLSLSSKLPLSSAFSVSLRPRHRGSRPLVLVV...,KKGVRIVNVA


In [41]:
sequences = df['Extracted_Sequence'].tolist()

In [38]:
import csv

# Specify the file path
csv_file = 'sequences.csv'

# Write the list to the CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sequence'])  # Write the header
    writer.writerows([[sequence] for sequence in sequences])  # Write each sequence as a row
